In [1]:

import numpy as np
import pandas as pd

In [2]:
from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'keras'

In [ ]:
# define constant & read the dataset
FOLDER = 'dataset/'
FILENAME = 'training_dataset.csv'

df = pd.read_csv(f'{FOLDER}{FILENAME}')
df['matchdate']=pd.to_datetime(df['matchday'], format='%Y-%m-%d')

df.shape

In [ ]:
df.sample(6)

In [ ]:
features = ['1N2_1','1N2_N','1N2_2','away_team_rank_FIFA','home_team_rank_FIFA']
target = ['home_score','away_score']
for idx, r in df.iterrows():
    s = r.correct_score.split(':')
    df.loc[idx,'home_score']=int(s[0])
    df.loc[idx,'away_score']=int(s[1]) 
df.drop('correct_score', axis=1, inplace=True)    

trainset = df.query('competition != "world-cup" or (competition == "world-cup" and season < 2014)').drop('competition', axis=1)[features+target]
testset = df.query('competition == "world-cup" and (season <= 2014)').drop('competition', axis=1)[features+target]
unseenset = df.query('competition == "world-cup" and (season == 2018)').drop('competition', axis=1)[features+target]

wc2018 =df.query('competition == "world-cup" and (season == 2018)')

trainset.shape, testset.shape, unseenset.shape


In [ ]:
trainset.sample(4)

In [ ]:
train_X = trainset.drop(['home_score','away_score'],axis=1)
train_y = trainset[['home_score', 'away_score']]


test_X = testset.drop(['home_score','away_score'],axis=1)
test_y = testset[['home_score', 'away_score']]


unseen_X = unseenset.drop(['home_score','away_score'],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(train_X)

train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)
unseen_X = scaler.transform(unseen_X)

In [ ]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(32, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mae',optimizer='adam')
    return model


In [ ]:
n_inputs = train_X.shape[1]
n_outputs = train_y.shape[1]


In [ ]:
model = get_model(n_inputs, n_outputs)

In [ ]:
model.fit(train_X, train_y, verbose=0, epochs=100)

In [ ]:
model.evaluate(x=test_X, y=test_y)

In [ ]:
model.predict(unseen_X[0:1,:])

In [ ]:
import joblib

#save your model or results
joblib.dump(model,'models/multi-regressor-score.pkl')

#load your model for further usage
model_loaded = joblib.load("models/multi-regressor-score.pkl")

In [ ]:
model_loaded.predict(unseen_X.iloc[0:1,:])

In [ ]:
print(unseen_X.iloc[0:1,:], unseen_X.iloc[0:1,:].dtypes)

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input


In [ ]:
def build_model():
    input_layer = Input(shape=(len(train_X.columns),))
    first_dense = Dense(units='128', activation='relu')(input_layer)
    y1_output = Dense(units='1', name='home_score')(first_dense)
    second_dense = Dense(units='128', activation='relu')(first_dense)
    y2_output=Dense(units='1', name='away_score')(second_dense)
    model = Model(inputs=input_layer, outputs=[y1_output, y2_output])
    return model




In [ ]:
model = build_model()

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001)

In [ ]:
model.compile(optimizer=optimizer,
    loss={'home_score_output':'mse','away_score_output':'mse'},
    metrics={'home_score_output':keras.metrics.RootMeanSquaredError(),
    'away_score_output':keras.metrics.RootMeanSquaredError()})
    

In [ ]:
history = model.fit(train_X, train_y, epochs=100, batch_size=10, validation_data=(test_X, test_y))


In [ ]:
train_X